In [2]:
import polars as pl
import numpy as np

In [3]:
gemma_path = "./data/doc_features/gemma_2b_cls/transcript_componenttext_{year}_{part}_cls_mean_meta.csv"
qwen_path = "./data/doc_features/qwen_4b_cls/transcript_componenttext_{year}_{part}_cls_mean_meta.csv"
llama_path = "./data/doc_features/llama3_3b_cls/transcript_componenttext_{year}_{part}_cls_mean_meta.csv"

gemma_npz_path = "./data/doc_features/gemma_2b_cls{new}/transcript_componenttext_{year}_{part}_cls_mean.npz"
qwen_npz_path = "./data/doc_features/qwen_4b_cls{new}/transcript_componenttext_{year}_{part}_cls_mean.npz"
llama_npz_path = "./data/doc_features/llama3_3b_cls{new}/transcript_componenttext_{year}_{part}_cls_mean.npz"
# year = 2012
# part = 1

# gemma_df = pl.read_csv(gemma_path.format(year=year, part=part))
# qwen_df = pl.read_csv(qwen_path.format(year=year, part=part))
# llama_df = pl.read_csv(llama_path.format(year=year, part=part))

In [4]:
for year in [2012, 2013, 2014]:
    for part in [1, 2]:
        gemma_df = pl.read_csv(gemma_path.format(year=year, part=part))
        qwen_df = pl.read_csv(qwen_path.format(year=year, part=part))
        llama_df = pl.read_csv(llama_path.format(year=year, part=part))   

        intersect_df = (
            gemma_df
            .join(qwen_df.select("transcriptid"), on="transcriptid", how="inner")
            .join(llama_df.select("transcriptid"), on="transcriptid", how="inner")
        )

        intersect_df.write_csv("./data/doc_features/{year}_{part}_master_metadata.csv".format(year=year, part=part))

        keep_ids = set(intersect_df['transcriptid'])

        for path in [gemma_npz_path, qwen_npz_path, llama_npz_path]:
            data = np.load(path.format(new="", year=year, part=part))
            X_cls       = data['X_cls']      
            X_mean      = data['X_mean']   
            transcriptids = data['transcriptids'] 

            mask = np.isin(transcriptids, list(keep_ids))

            X_cls_filt       = X_cls[mask]
            X_mean_filt      = X_mean[mask]
            transcriptids_filt = transcriptids[mask]

            np.savez(
                path.format(new="_filtered", year=year, part=part),
                X_cls=       X_cls_filt,
                X_mean=      X_mean_filt,
                transcriptids=transcriptids_filt
            )


FileNotFoundError: No such file or directory (os error 2): ./data/doc_features/gemma_2b_cls/transcript_componenttext_2012_1_cls_mean_meta.csv